# SEN163A - Fundamentals of Data Analytics
# Assignment 2 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 05-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
short description of the assignment

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [2]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas

In [1]:
#AS Dataset

AS_Filename = 'data/AS_dataset.pkl'

with open(AS_Filename, 'rb') as file:
    
    AS_df = pickle.load(file)

In [ ]:
#Probe dataset

Probe_Filename = 'data/probe_dataset.pkl'

with open(Probe_Filename, 'rb') as file:
    
    AS_df = pickle.load(file)

In [ ]:
#Ripe dataset 

#Method needs to be decided

In [3]:
#IP 2 Location dataset

IP_Filename = "data/IP2LOCATION-LITE-DB1.CSV"

ipv4_df = pandas.read_csv(IP_Filename)

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

...

In [ ]:
#Code needed to prove limitations

Some list of limitations in text

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 AS

With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last
three in your report (number, name and country).


In [5]:
#Code for finding the list of AS

#Printed list of the AS sorted like mentioned above

Description of results

## 2.2 Hosting location
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way.

In [6]:
#Code for finding all host probes from EU in the dataset of one hour

Description of results

## 2.3 Latency
Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one ncountries ×m matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)


In [7]:
#Code...

Description of results

## 2.4 Optimal server locations
Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.


In [8]:
#Code...

Description of results

# Conclusions

... 
add code if needed